In [1]:
# load basic packages
import os
import sys
import warnings

import re
import numpy as np
import pandas as pd
import scipy as sp
import json

import matplotlib.pyplot as plt
plt.style.use("seaborn")
import seaborn as sns
import spacy
nlp = spacy.load("en")

# save model
import pickle

# set environment
pd.set_option("display.max_columns", 100)
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("data/train.csv")

In [3]:
data.columns

Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'socialEngagementType', 'totals', 'trafficSource',
       'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')

In [4]:
data.head(3)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386


### channelGrouping

In [5]:
data.channelGrouping.unique()

array(['Organic Search', 'Referral', 'Paid Search', 'Affiliates',
       'Direct', 'Display', 'Social', '(Other)'], dtype=object)

In [6]:
print(data.channelGrouping.isnull().sum(), data.channelGrouping.isna().sum())

0 0


In [7]:
channelGroupingMap = {
    "Organic Search": 1,
    "Referral": 2,
    "Paid Search": 3,
    "Affiliates": 4,
    "Direct": 5,
    "Display": 6,
    "Social": 7,
    "(Other)": 8
}

In [8]:
data.channelGrouping.dtypes

dtype('O')

In [9]:
data.channelGrouping = data.channelGrouping.replace(channelGroupingMap)
data.channelGrouping = data.channelGrouping.astype("category")

In [10]:
data.head(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,1,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,1,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


### socialEngagementType

In [11]:
data.socialEngagementType.unique()

array(['Not Socially Engaged'], dtype=object)

In [12]:
print(data.socialEngagementType.isnull().sum(), data.socialEngagementType.isna().sum())

0 0


In [13]:
data = data[[col for col in data.columns if col != "socialEngagementType"]]
data.columns

Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'totals', 'trafficSource', 'visitId', 'visitNumber',
       'visitStartTime'],
      dtype='object')

### visitId

In [14]:
data.visitId.unique()

array([1472830385, 1472880147, 1472865386, ..., 1483526434, 1483592857,
       1483574474])

In [15]:
data.visitId.isnull().sum()

0

In [16]:
data.visitId.isna().sum()

0

### visitStartTime

In [17]:
data.visitStartTime.unique()

array([1472830385, 1472880147, 1472865386, ..., 1483526434, 1483592864,
       1483574474])

In [18]:
data.visitStartTime.isnull().sum()

0

In [19]:
data.visitStartTime.isna().sum()

0

### visitNumber

In [20]:
data.visitNumber.unique()

array([  1,   2,   3,   5,  11,   4,  57,   6,  56,   7,  20,   8,  15,
         9,  25,  24,  14,  89, 136,  13,  85,  10, 105,  22,  21,  48,
        18,  26,  17,  83,  38,  84,  27,  42, 100,  31,  16,  30,  59,
        50,  19,  51,  29, 160,  52,  12,  63,  23,  47,  49,  28,  34,
       178,  88,  33, 317,  93, 140,  35,  41,  91,  92,  74, 309,  43,
        61,  46,  45,  32, 108,  86, 195,  36, 236,  72, 162, 235,  90,
        76,  77,  94,  96,  73,  78,  97,  87, 304, 106, 107, 389,  68,
        98, 132,  58, 266, 267, 221,  39, 121, 143, 142, 174, 175, 189,
       191,  55, 190, 104, 245, 204,  37,  82, 137, 206, 101,  62, 110,
       156, 158, 157, 159, 138, 109, 194, 193, 147, 254,  71,  53, 144,
       145, 146,  80, 253,  99, 134, 135, 102,  44, 130,  95,  75, 103,
       315,  70,  69, 117, 163, 154, 153, 283,  65, 262, 263, 295, 297,
       298, 219,  64, 296, 155, 141, 169, 170,  60, 207,  54, 187, 133,
       118, 264,  67,  66, 126,  40, 122, 325, 326, 197, 167, 19

In [21]:
data.visitNumber.isnull().sum()

0

In [22]:
data.visitNumber.isna().sum()

0

### totals

In [23]:
data["transactionRevenue"] = pd.DataFrame(data.totals.apply(json.loads).tolist())[["transactionRevenue"]]
data.head(2)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,totals,trafficSource,visitId,visitNumber,visitStartTime,transactionRevenue
0,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385,NaN
1,1,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147,NaN


In [24]:
data.shape

(903653, 12)

In [25]:
data = data.dropna(subset=["transactionRevenue"])

In [26]:
data.shape

(11515, 12)

In [27]:
data.head(2)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,totals,trafficSource,visitId,visitNumber,visitStartTime,transactionRevenue
752,5,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6194193421514403509,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",6194193421514403509_1472843572,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""11...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1472843572,1,1472843572,37860000
753,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5327166854580374902,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",5327166854580374902_1472844906,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""10...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472844906,3,1472844906,306670000


In [28]:
device_json_dump = data.device.apply(json.loads)

In [29]:
type(device_json_dump)

pandas.core.series.Series

In [30]:
device_json_dump.iloc[0]

{'browser': 'Chrome',
 'browserVersion': 'not available in demo dataset',
 'browserSize': 'not available in demo dataset',
 'operatingSystem': 'Linux',
 'operatingSystemVersion': 'not available in demo dataset',
 'isMobile': False,
 'mobileDeviceBranding': 'not available in demo dataset',
 'mobileDeviceModel': 'not available in demo dataset',
 'mobileInputSelector': 'not available in demo dataset',
 'mobileDeviceInfo': 'not available in demo dataset',
 'mobileDeviceMarketingName': 'not available in demo dataset',
 'flashVersion': 'not available in demo dataset',
 'language': 'not available in demo dataset',
 'screenColors': 'not available in demo dataset',
 'screenResolution': 'not available in demo dataset',
 'deviceCategory': 'desktop'}

Based on domain knowledge, this factor will not affect the revenue of the customer.

In [31]:
data.columns

Index(['channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
       'sessionId', 'totals', 'trafficSource', 'visitId', 'visitNumber',
       'visitStartTime', 'transactionRevenue'],
      dtype='object')

### geoNetwork

In [32]:
data.geoNetwork.apply(json.loads).iloc[:3]

752    {'continent': 'Americas', 'subContinent': 'Nor...
753    {'continent': 'Americas', 'subContinent': 'Nor...
799    {'continent': 'Americas', 'subContinent': 'Nor...
Name: geoNetwork, dtype: object

In [33]:
data["country"] = data.geoNetwork.apply(json.loads).apply(lambda x: x["country"])
data["continent"] = data.geoNetwork.apply(json.loads).apply(lambda x: x["continent"])
data["region"] = data.geoNetwork.apply(json.loads).apply(lambda x: x["region"])

In [34]:
for x in ["country", "region", "continent"]:
    print("isnull:", data[[x]].isnull().sum())
    print("isna:", data[[x]].isna().sum())

isnull: country    0
dtype: int64
isna: country    0
dtype: int64
isnull: region    0
dtype: int64
isna: region    0
dtype: int64
isnull: continent    0
dtype: int64
isna: continent    0
dtype: int64


In [35]:
region_labels = list(data.region.unique())
region_map = dict()
i = 1
for reg in region_labels:
    region_map[reg] = i
    i += 1

In [36]:
country_labels = list(data.country.unique())
country_map = dict()
i = 1
for cn in country_labels:
    country_map[cn] = i
    i += 1

In [37]:
continent_labels = list(data.continent.unique())
continent_map = dict()
i = 1
for cn in continent_labels:
    continent_map[cn] = i
    i += 1

In [38]:
data.continent = data.continent.replace(continent_map)
data.country = data.country.replace(country_map)
data.region = data.region.replace(region_map)

In [39]:
data.continent = data.continent.astype("category")
data.country = data.country.astype("category")
data.region = data.region.astype("category")

In [40]:
data.head(3)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,totals,trafficSource,visitId,visitNumber,visitStartTime,transactionRevenue,country,continent,region
752,5,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6194193421514403509,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",6194193421514403509_1472843572,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""11...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1472843572,1,1472843572,37860000,1,1,1
753,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5327166854580374902,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",5327166854580374902_1472844906,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""10...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472844906,3,1472844906,306670000,1,1,2
799,2,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8885051388942907862,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",8885051388942907862_1472827393,"{""visits"": ""1"", ""hits"": ""13"", ""pageviews"": ""11...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1472827393,7,1472827393,68030000,1,1,2


### tarrifSource

In [41]:
data.trafficSource.apply(json.loads).iloc[:10]

752    {'campaign': '(not set)', 'source': '(direct)'...
753    {'campaign': '(not set)', 'source': 'google', ...
799    {'referralPath': '/', 'campaign': '(not set)',...
802    {'referralPath': '/', 'campaign': '(not set)',...
859    {'referralPath': '/', 'campaign': '(not set)',...
866    {'referralPath': '/', 'campaign': '(not set)',...
893    {'campaign': '(not set)', 'source': '(direct)'...
910    {'referralPath': '/', 'campaign': '(not set)',...
922    {'campaign': '(not set)', 'source': 'google', ...
925    {'referralPath': '/a/google.com/google-merchan...
Name: trafficSource, dtype: object

In [42]:
data["source"] = data.trafficSource.apply(json.loads).apply(lambda x: x["source"].strip("[.,()!?']"))
data.head(3)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,totals,trafficSource,visitId,visitNumber,visitStartTime,transactionRevenue,country,continent,region,source
752,5,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6194193421514403509,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",6194193421514403509_1472843572,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""11...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1472843572,1,1472843572,37860000,1,1,1,direct
753,1,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5327166854580374902,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",5327166854580374902_1472844906,"{""visits"": ""1"", ""hits"": ""11"", ""pageviews"": ""10...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472844906,3,1472844906,306670000,1,1,2,google
799,2,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8885051388942907862,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",8885051388942907862_1472827393,"{""visits"": ""1"", ""hits"": ""13"", ""pageviews"": ""11...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1472827393,7,1472827393,68030000,1,1,2,mall.googleplex.com


In [43]:
# remove duplicate columns
filter_columns = ["device", "geoNetwork", "totals", "trafficSource"]

In [44]:
data = data[[col for col in data.columns if col not in filter_columns]]
data.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'transactionRevenue', 'country',
       'continent', 'region', 'source'],
      dtype='object')

In [45]:
data.head(5)

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,transactionRevenue,country,continent,region,source
752,5,20160902,6194193421514403509,6194193421514403509_1472843572,1472843572,1,1472843572,37860000,1,1,1,direct
753,1,20160902,5327166854580374902,5327166854580374902_1472844906,1472844906,3,1472844906,306670000,1,1,2,google
799,2,20160902,8885051388942907862,8885051388942907862_1472827393,1472827393,7,1472827393,68030000,1,1,2,mall.googleplex.com
802,2,20160902,0185467632009737931,0185467632009737931_1472846398,1472846398,6,1472846398,26250000,1,1,3,mall.googleplex.com
859,2,20160902,3244885836845029978,3244885836845029978_1472824817,1472824817,4,1472824817,574150000,1,1,4,mall.googleplex.com


In [46]:
data.source.unique()

array(['direct', 'google', 'mall.googleplex.com', 'sites.google.com',
       'dfa', 'mail.google.com', 'gdeals.googleplex.com',
       'l.facebook.com', 'youtube.com', 'facebook.com', 'dealspotr.com',
       'mail.aol.com', 'bing', 'groups.google.com', 'yahoo',
       'connect.googleforwork.com', 'phandroid.com', 'duckduckgo.com',
       'reddit.com', 'googleux.perksplus.com', 'pinterest.com',
       'trainup.withgoogle.com', 'search.xfinity.com', 't.co',
       'google.com', 'plus.google.com', 'keep.google.com',
       'l.messenger.com', 'ask', 'quora.com', 'mg.mail.yahoo.com',
       'm.facebook.com', 'siliconvalley.about.com', 'Partners',
       'search.myway.com', 'chat.google.com', 'docs.google.com',
       'seroundtable.com', 'outlook.live.com', 'basecamp.com',
       'gatewaycdi.com', 'calendar.google.com', 'moma.corp.google.com',
       'us-mg5.mail.yahoo.com'], dtype=object)

In [47]:
labels = list(data.source.unique())

In [48]:
source_map = dict()
i = 1
for lab in labels:
    if lab not in source_map:
        source_map[lab] = i
        i += 1
    else:
        continue
# source_map

In [49]:
data.source = data.source.replace(source_map)
data.source = data.source.astype("category")
data.head(5)

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,transactionRevenue,country,continent,region,source
752,5,20160902,6194193421514403509,6194193421514403509_1472843572,1472843572,1,1472843572,37860000,1,1,1,1
753,1,20160902,5327166854580374902,5327166854580374902_1472844906,1472844906,3,1472844906,306670000,1,1,2,2
799,2,20160902,8885051388942907862,8885051388942907862_1472827393,1472827393,7,1472827393,68030000,1,1,2,3
802,2,20160902,0185467632009737931,0185467632009737931_1472846398,1472846398,6,1472846398,26250000,1,1,3,3
859,2,20160902,3244885836845029978,3244885836845029978_1472824817,1472824817,4,1472824817,574150000,1,1,4,3


In [50]:
data.dtypes

channelGrouping       category
date                     int64
fullVisitorId           object
sessionId               object
visitId                  int64
visitNumber              int64
visitStartTime           int64
transactionRevenue      object
country               category
continent             category
region                category
source                category
dtype: object

In [51]:
data.to_csv("data/cleaned_train.csv", index=False)

In [52]:
df = pd.read_csv("data/cleaned_train.csv")
df.head(5)

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,transactionRevenue,country,continent,region,source
0,5,20160902,6194193421514403509,6194193421514403509_1472843572,1472843572,1,1472843572,37860000,1,1,1,1
1,1,20160902,5327166854580374902,5327166854580374902_1472844906,1472844906,3,1472844906,306670000,1,1,2,2
2,2,20160902,8885051388942907862,8885051388942907862_1472827393,1472827393,7,1472827393,68030000,1,1,2,3
3,2,20160902,185467632009737931,0185467632009737931_1472846398,1472846398,6,1472846398,26250000,1,1,3,3
4,2,20160902,3244885836845029978,3244885836845029978_1472824817,1472824817,4,1472824817,574150000,1,1,4,3
